# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.train.automl import AutoMLConfig


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {cluster_name}")
except ComputeTargetException:
    print(f"Creating new compute target: {cluster_name}")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_D2_V2",
        min_nodes=0,
        max_nodes=4
    )

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

In [ ]:
# !pip install kagglehub[pandas-datasets]

In [7]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "temp.csv"
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "bls/denver-cpi",
  file_path,
)

C:\Users\Aseer\AppData\Local\Temp\ipykernel_20628\2095982378.py:5: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


KaggleApiHTTPError: 404 Client Error.

Resource not found at URL: https://www.kaggle.com/datasets/bls/denver-cpi/versions/1
The server reported the following issues: Data not found
Please make sure you specified the correct resource identifiers.

In [ ]:
dataset = None
import kagglehub
from kagglehub import KaggleDatasetAdapter
from azureml.core import Dataset
from azureml.data.dataset_profile import DataProfile
from azureml.data.dataset_profile.run_config import DatasetProfileRunConfig

file_path = ""
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "bls/denver-cpi",
  file_path,
)

datastore = ws.get_default_datastore()

ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe = df, 
        name='denver-cpi',
        description='CPI denver dataset',
        target=datastore
    )

profile_run_config = DatasetProfileRunConfig(compute_target=compute_target)
data_profile = DataProfile(profile_run_config)
profile_result = data_profile.profile(ds)



## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes" : 5,
    "max_concurrent_iterations" : 3,
    "primary_metric" : '',
    "verbosity" : logging.INFO
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data = dataset,
    label_column_name="y",
    n_cross_validations=3,
    compute_target=compute_target,
    **automl_settings)

In [ ]:
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run = remote_run.get_best_run_by_primary_metric()


In [ ]:
import joblib

# best_run.download_file(name='outputs/model.joblib', output_file_path='best_model.joblib')
# model = joblib.load('best_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
